In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
# basic prompting

from langchain_teddynote.messages import stream_response
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 맞게 [FORMAT]에 영어 회화를 작성해 주세요.

상황:
{question}

FORMAT:
- 영어 회화:
- 한글 해석:
"""

prompt = PromptTemplate.from_template(template)

model = ChatOpenAI(
    model_name="gpt-3.5-turbo"
)

output_parser = StrOutputParser()

chain = prompt | model | output_parser

answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요."})
stream_response(answer)

- 영어 회화: Can I take your order?
- 한글 해석: 주문을 받아도 될까요?

In [ ]:
# batch prompting

from langchain_teddynote.messages import stream_response
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template("{topic}에 대해서  3문장으로 알려줘.")

model = ChatOpenAI()

chain = prompt | model | StrOutputParser()

chain.batch([
    {"topic": "ChatGPT"},
    {"topic": "Instagram"},
    {"topic": "멀티모달"},
    {"topic": "프로그래밍"},
    {"topic": "머신러닝"}
],
config = {"max_concurrency": 3},
)

['ChatGPT는 OpenAI가 개발한 자연어 처리 기반 대화 모델로, 사용자와 자연스럽게 대화할 수 있는 인공지능 챗봇이다. 이 모델은 대규모의 데이터셋을 학습하여 다양한 주제에 대해 대화를 이어나갈 수 있으며, 실제로 사람과의 대화와 거의 구분이 안 될 정도로 자연스러운 대화를 제공한다. ChatGPT는 다양한 분야에서 활용 가능하며, 문제 해결, 정보 제공, 상담, 엔터테인먼트 등 다양한 용도로 활용될 수 있다.',
 'Instagram은 사진과 동영상을 공유할 수 있는 소셜 미디어 플랫폼으로, 사용자들은 자신의 일상을 시각적으로 공유하고 다른 사람들과 소통할 수 있습니다. 해시태그를 통해 관심사나 주제에 맞는 콘텐츠를 찾을 수 있고, 팔로우와 좋아요를 통해 다른 사용자와 상호작용할 수 있습니다. 또한 인스타그램 스토리 기능을 통해 24시간 동안 임시로 사진이나 동영상을 공유할 수 있습니다.',
 '멀티모달은 여러 종류의 정보를 결합하여 제공하는 시스템이다. 이는 텍스트, 이미지, 음성, 동영상 등 다양한 매체를 활용하여 사용자에게 정보를 전달한다. 멀티모달은 사용자 경험을 향상시키고 효율적인 커뮤니케이션을 가능하게 한다.',
 '프로그래밍은 컴퓨터에게 명령을 내리는 일종의 언어이며, 이를 통해 원하는 기능을 구현할 수 있다. 프로그래밍을 배우면 문제 해결능력과 창의력이 향상되며, 다양한 분야에서 활용할 수 있는 능력을 기를 수 있다. 논리적 사고와 문제 해결능력을 갖춘 사람에게는 프로그래밍이 더욱 효율적이고 흥미로운 일이 될 것이다.',
 '머신러닝은 컴퓨터가 데이터를 학습하여 패턴을 발견하고 예측하는 인공지능 기술이다. 주어진 데이터를 기반으로 모델을 학습시켜 새로운 데이터에 대한 결정을 내릴 수 있다. 주로 분류, 회귀, 클러스터링 등의 작업에 활용된다.']

In [ ]:
# parallel prompting

from langchain_core.runnables import RunnableParallel

chain1 = (
    PromptTemplate.from_template("{country}의 수도는 어디야?") 
    | model
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country}의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

combined = RunnableParallel(capital=chain1, area=chain2)

combined.invoke({"country": "대한민국"})
combined.invoke("대한민국") # 1개의 변수만 템플릿에 포함되어 있다면 값만 전달하는 것도 가능함. # 원래는 RunnablePassthrough를 써서 했었어야 함. 

combined.batch([{"country": "미국"}, {"country": "대한민국"}])

[{'capital': '미국의 수도는 워싱턴 D.C.입니다.', 'area': '미국의 면적은 약 9,833,520 km² 입니다.'},
 {'capital': '대한민국의 수도는 서울이에요.', 'area': '대한민국의 총 면적은 약 100,363km²이다.'}]

In [ ]:
# runnable passthrough -> when passing arguments to a function, we don't have to use dictionary form. 

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

prompt = PromptTemplate.from_template("{num} 의 10배는?")

runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()

runnable_chain.invoke(10)

AIMessage(content='100입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 16, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0e0af6b1-3bfe-4912-9a81-4f145579801b-0', usage_metadata={'input_tokens': 16, 'output_tokens': 4, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# runnable lambda -> we can use user defined function

from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime

def get_today(a):
    return datetime.today().strftime("%b-%d")

prompt = PromptTemplate.from_template(
    "{today}가 생일인 유명인 {n}명을 나열하세요. 생년월일을 표기해 주세요."
)
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

chain= (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke(3)

'1. 엘비스 프레슬리 (Elvis Presley) - 1935년 1월 8일\n2. 데이비드 보위 (David Bowie) - 1947년 1월 8일\n3. 스티브 헤럴드 (Steve Harwell) - 1967년 1월 8일'

In [14]:
chain.invoke(input)

AIMessage(content='인공지능 모델 학습의 원리는 데이터를 입력으로 받아들이고 이를 처리하여 원하는 결과를 출력하는 과정입니다. 모델 학습은 크게 입력 데이터, 모델, 손실 함수, 최적화 알고리즘으로 구성됩니다.\n\n먼저 입력 데이터를 모델에 입력하면 모델은 입력 데이터를 처리하여 출력을 생성합니다. 이때 모델은 일련의 가중치와 편향을 가지고 있으며, 이들은 모델이 입력 데이터를 올바르게 처리할 수 있도록 조정되어야 합니다.\n\n손실 함수는 모델의 출력과 실제 정답 간의 차이를 측정하는 함수로, 모델이 얼마나 잘 동작하는지를 평가하는 데 사용됩니다. 목표는 손실 함수의 값을 최소화하는 것이며, 이를 위해 최적화 알고리즘을 사용하여 모델의 가중치와 편향을 조정합니다.\n\n이러한 과정을 반복하여 모델을 훈련시키면, 모델은 입력 데이터를 올바르게 처리하는 능력을 향상시키고, 새로운 데이터에 대해서도 정확한 결과를 출력할 수 있도록 학습됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 369, 'prompt_tokens': 33, 'total_tokens': 402, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-30d7c6f1-30d6-4bf7-911f-cb7555adfe97-0', usage_meta

In [15]:
answer = chain.stream(input)
stream_response(answer)

인공지능 모델 학습은 데이터를 입력으로 받아들여 일련의 계산과정을 통해 데이터의 패턴이나 관계를 학습하는 과정입니다. 이때 모델은 입력된 데이터를 기반으로 가중치를 업데이트하며 예측을 수행하게 됩니다.

먼저 학습 데이터를 입력으로 받아들이고, 모델은 입력 데이터를 분석하여 내부적으로 계산을 수행합니다. 이러한 과정을 통해 모델은 데이터의 특성을 학습하고, 학습 데이터에 대한 예측을 수행할 수 있게 됩니다.

이때 모델은 학습 데이터를 이용하여 가중치를 조정하고, 오차를 최소화하는 방향으로 학습을 진행합니다. 이렇게 학습된 모델은 새로운 데이터에 대해 예측을 수행할 수 있으며, 이를 통해 다양한 분야에서 활용될 수 있습니다.

In [16]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | model | output_parser

input = {"topic": "인공지능 모델의 학습 원리"}

chain.invoke(input)

'인공지능 모델의 학습 원리는 데이터를 입력하여 모델이 패턴을 학습하고 예측을 만들어내는 과정입니다. 모델은 입력된 데이터를 분석하고 특징을 추출하여 내부적인 구조를 학습하게 됩니다. 이때 모델은 일정한 목표나 기준에 따라서 학습을 진행하며, 이를 위해 학습 데이터와 정답 데이터를 이용하여 오차를 최소화하도록 최적화 알고리즘을 적용합니다. 이 과정을 반복하면서 모델은 점차 정확도를 향상시키게 되어 원하는 작업을 수행할 수 있게 됩니다.'

In [17]:
answer = chain.stream(input)

stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 입력으로 받아들이고 이를 처리하여 원하는 결과를 출력하는 과정입니다. 이 모델은 입력 데이터와 출력 데이터 사이의 관계를 학습하여 예측이나 분류를 수행합니다.

모델은 학습을 통해 데이터의 패턴을 파악하고, 입력 데이터와 출력 데이터 사이의 관계를 이해합니다. 학습 과정에서 모델은 손실 함수를 최소화하는 방향으로 가중치를 업데이트하며 최적의 결과를 찾아나갑니다.

이러한 학습 원리를 통해 인공지능 모델은 주어진 데이터를 기반으로 새로운 데이터에 대한 예측을 수행할 수 있게 됩니다.